# Kostya's stuff

In [1]:
%pylab inline
%run ../utils.py
%run ../3DVariables/Functions.py

Populating the interactive namespace from numpy and matplotlib
import failed

In [2]:
data = get_data()

NameError: name 'get_data' is not defined

In [3]:
from matplotlib.colors import LogNorm
import seaborn as s
s.set(rc={'image.cmap': "jet"})
config_plots(rcParams)

In [4]:
sel = lambda s: [ c for c in data.columns 
    if c.startswith(s) and c[-1] not in ["0","8"] ]

etrack = data[sel("EDepLayer")].mean(axis=1)
scatter(data['BetaTOF'], etrack, c='k', s=1, marker='.', lw=0)
plot(Beta_cent,ETrack,'b')
plot(Beta_cent,ETrack+sigmaEtrackinv*(ETrack**2),'r')
plot(Beta_cent,ETrack-sigmaEtrackinv*(ETrack**2),'r')
xlim(0.4,1.0); ylim(0,1)
xlabel("Beta TOF"); ylabel("mean EdepLayer in inner")

NameError: name 'data' is not defined

In [5]:
sel = lambda s: [ c for c in data.columns if c.startswith(s) ]

scatter(data['BetaTOF'], data[sel('EdepTOF')].mean(axis=1), c='k', s=1, marker='.', lw=0)
plot(Beta_cent,ETOF,'b')
plot(Beta_cent,ETOF+sigmaEtofinv*(ETOF**2),'r')
plot(Beta_cent,ETOF-sigmaEtofinv*(ETOF**2),'r')
xlim(0.4,1.0); ylim(0,10)
xlabel("Beta TOF"); ylabel("mean EdepTOF")

NameError: name 'data' is not defined

In [6]:
scatter(data['BetaTOF'], data['EdepTRD']/data['NTRDclusters'], c='k', s=1, marker='.', lw=0)
plot(Beta_cent,ETRD,'b')
plot(Beta_cent,ETRD+sigmaETRDinv*(ETRD**2),'r')
plot(Beta_cent,ETRD-sigmaETRDinv*(ETRD**2),'r')
xlim(0.4,1.0); ylim(0,10)
xlabel("Beta TOF"); ylabel("EdepTRD/NTRDclusters")

NameError: name 'data' is not defined

In [7]:
scatter(data['R'],data['BetaTOF'], c='k', s=1, marker='.', lw=0)
Rs = valorecent
Bs = Rs/np.sqrt(1.875**2+Rs**2)
plot(Rs,Bs,'b')
plot(Rs + sigmaRinv*(Rs**2),Bs,'r')
plot(Rs - sigmaRinv*(Rs**2),Bs,'r')
xlim(0,10); ylim(0.4,1)

NameError: name 'data' is not defined

In [8]:
scatter(data['R'],data['BetaTOF'], c='k', s=1, marker='.', lw=0)
Bs = betacent 
Rs = 1.875*Bs/np.sqrt(1-Bs**2)
plot(Rs,Bs,'b')
plot(Rs,Bs + sigmabetainv*(Bs**2),'r')
plot(Rs,Bs - sigmabetainv*(Bs**2),'r')
xlim(0,10); ylim(0.4,1)

NameError: name 'data' is not defined

In [9]:
Beta_cent

NameError: name 'Beta_cent' is not defined

# Benoît's stuff

## Comparison between observed target matrix and the MCMC predicted matrix

The following plot is the target matrix obtained with real data.
The cut used are:
[
    "physicsTrigger",
    "chargeOne",
    "oneTrack",
    "goldenTOF",
    "goldenTRACKER",
    "oneParticle",
    "goldenTRD",
    "betaNotCrazy"]

x axis is rigidity measured and y axis is beta measured


<img src="files/protonObserved.png">

The following plot is the MCMC predicted matrix taking into account only protons (ie. the deuton flux is set to 0). Is is obtained by taking the proton parameters where the MCMC has converged and build the expected beta Vs rig distribution with them. **It is based on AMS.protonsB800**

<img src="files/protonPredicted.png">

Let's look at the difference between the predicted distribution and the observed one.
The following plog is : (predicted - observed)/observed

<img src="files/protonPredictedMinusObserved.png">

The enveloppes added to the plot represents the observed distribution
We are not interested by the red blob which is a region that will be masked. We are only interested by the region where the enveloppes are.

What we can see is **there is an excess of proton in the peak of the distribution and there is a lack of proton in the tail. This can be explained by the fact that the sigma of beta TOF in the Monte-Carlo is too small.**

(incidently we can also see the blue region where the deutons are missing. It is normal since no deutons where injected in the MCMC)

## investigating sigma

The following plot shows the mean value of beta TOF as a function of the measured rigidity for data and MC. It is obtained by vertically slicing the aforementioned hObserved and hPredicted distributions and fitting every slice by a gaussian. The mean of every gaussian is the y-value of every point.
<img src="files/betaTOFMeanVsRig.png">
The agreement between both graphs is good.

However this is not the case for the sigma distribution.
<img src="files/betaTOFSigmaVsRig.png">

# onclusion

**At least in protonsB800 the beta TOF sigma is wrong by ~12%**

In [2]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import glob

%pylab inline

import os


Populating the interactive namespace from numpy and matplotlib


In [75]:
def loadBinary(dirname):
    print 'loading : ' + dirname
    data=dict()
    for file in os.listdir(dirname):
        if file.startswith(('time_L','R_','betaH_','beta_','NTofClusterH_')):
            data[file.split('_chunk')[0]] = np.fromfile(dirname+'/'+file, np.dtype('float32'))

    return DataFrame(data)

def makeDataframe(files, name):
    df = pd.concat(map(loadBinary, files[:50]))
    df = df[(df.time_L0 > -999) & (df.time_L1 > -999) & (df.time_L2 > -999) & (df.time_L3 > -999) & (df.R < 15) & (df.R > 1) & (df.betaH > 0.5) & (df.betaH < 1.2) & (df.NTofClusterH == 4)]
    df['binRgdt'] = df['R'].astype('int')
    df.index.name = name
    return df

dfMC = makeDataframe( glob.glob('/afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/protons.B1034_pr.pl1.1200.qgsp_bic_ams/*.00000001.output'), "MC" )
dfData = makeDataframe( glob.glob('/afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/ISS.B950_pass6/*.00000001.output'), "Data" )


    


loading : /afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/ISS.B950_pass6/1306219522.00000001.output


loading : /afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/ISS.B950_pass6/1306219312.00000001.output


loading : /afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/ISS.B950_pass6/1306218508.00000001.output


loading : /afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/ISS.B950_pass6/1306217884.00000001.output


loading : /afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/ISS.B950_pass6/1306217233.00000001.output

...


loading : /afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/protons.B1034_pr.pl1.1200.qgsp_bic_ams/872731550.00000001.output


loading : /afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/protons.B1034_pr.pl1.1200.qgsp_bic_ams/872731549.00000001.output


loading : /afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/protons.B1034_pr.pl1.1200.qgsp_bic_ams/872731548.00000001.output


loading : /afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/protons.B1034_pr.pl1.1200.qgsp_bic_ams/872731547.00000001.output

loading : /afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/protons.B1034_pr.pl1.1200.qgsp_bic_ams/1946159363.00000001.output
loading : /afs/cern.ch/user/b/bcoste/myeos/dstTofTiming/protons.B1034_pr.pl1.1200.qgsp_bic_ams/1946159364.00000001.output

In [69]:
import sys
import bigQueryPlotting as bq
import math

def binning(df,var,nBins,firstBin,lastBin):
    binWidth=(lastBin-firstBin)/nBins
    return firstBin + binWidth * (df[var]-firstBin).floordiv(binWidth)

def plotDataFrame2D(df,nBinsX,firstBinX,lastBinX,nBinsY,firstBinY,lastBinY,varX,varY):
    binX='bin_'+varX
    binY='bin_'+varY
    dfTmp=pd.DataFrame(df[[varX,varY]])
    dfTmp[binX]=binning(dfTmp,varX,nBinsX,firstBinX,lastBinX)
    dfTmp[binY]=binning(dfTmp,varY,nBinsY,firstBinY,lastBinY)
    dfTmp=dfTmp[(dfTmp[binX] >= firstBinX) & (dfTmp[binX] < lastBinX) & (dfTmp[binY] >= firstBinY) & (dfTmp[binY] < lastBinY) ]
    plot_matrix( dfTmp.groupby([binY,binX]).count()[varX].unstack().fillna(0) )

for df in [dfData,dfMC]:
    print df.index.name
    df['invBetaH']=1/df.betaH
    df['time_L3-time_L1']=df['time_L3']-df['time_L1']
    figure()
    plotDataFrame2D(df,100,0.9,1.6,100,2,7.,'invBetaH','time_L3-time_L1')




Data
MC

Beta distribution of MC and data is similar when integrated over the rigidity

In [70]:
for df in [dfData,dfMC]:
    df['betaH'].hist(normed=True,range=(0.8,1.1),bins=100,histtype='step')

However if we look per rigidity slice, this is not the case:

In [71]:
def makeHistAndFit(data,init,hist_range,funcFit=None,**kwargs):
    if funcFit is None: funcFit=gaussian
    n, bins, patches = plt.hist(data,normed=True,range=hist_range,bins=100,histtype='step',linewidth=2.0,**kwargs)
    ylim(ymin=0)
    xdata, ydata = ((bins[:-1] + bins[1:])*0.5,n)
    out = fit(funcFit,xdata,ydata,init)
    x,y = zip(*filter(lambda x: -out[0][2] < x[0]-out[0][1] < out[0][2],zip(xdata,ydata)))
    out = fit(funcFit,x,y,out[0])
    plt.plot(x,funcFit(out[0],x),linewidth=2.0)
    return (out[0][1],out[0][2])

def fitAllRigBin(data,var,init,hist_range,**kwargs):
    par = []
    for rig in rigBins:
        name=var + 'distribution - rigidity : ['+str(rig)+','+str(rig+1)+']'
        figure(name)
        title(name)
        par.append( makeHistAndFit(df[df['binRgdt']==rig][var].values, init, hist_range,**kwargs) )
    return par




### betaH distributions

In [72]:
mean = dict()
std = dict()

variables={}

variables['betaH']=([1.0, 0.5, 0.5], (0.6, 1.1))
mean['betaH']=dict()
std['betaH']=dict()
for df in [dfData,dfMC]:
    mean['betaH'][df.index.name],std['betaH'][df.index.name]=zip(* fitAllRigBin(df,'betaH',variables['betaH'][0],variables['betaH'][1]) )    



### T3-T1 distributions

In [73]:
variables['time_L3-time_L1']=([1.0, 6.0, 1.0], (2,8))
mean['time_L3-time_L1']=dict()
std['time_L3-time_L1']=dict()
for df in [dfData,dfMC]:
    mean['time_L3-time_L1'][df.index.name],std['time_L3-time_L1'][df.index.name]=zip(* fitAllRigBin(df,'time_L3-time_L1',variables['time_L3-time_L1'][0],variables['time_L3-time_L1'][1]) )    

In [74]:
i=0
figure(figsize=(10,10))
for var in variables:
    subplot(221+i)
    plt.title('Mean '+ var + ': ratio Data/MC')
    plt.plot(rigBins,[x/y for x,y in zip(mean[var]['Data'],mean[var]['MC'])],'o')
    i+=1

    subplot(221+i)
    plt.title('Standard deviation '+ var + ': ratio Data/MC')
    plt.plot(rigBins,[x/y for x,y in zip(std[var]['Data'],std[var]['MC'])],'o')
    i+=1

